<a href="https://colab.research.google.com/github/jae-gyeong/marchine_learning_start/blob/master/19_10_02cnn(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [0]:
X = tf.placeholder(tf.float32, [None,28,28,1])
Y = tf.placeholder(tf.float32, [None,10])

## CNN 계층구성

In [0]:
# conv - (relu) - pooling
# conv - (relu) - pooling
# FC

In [9]:
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01)) # 표준편차 0.01 32개필터 지정 W1커널
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1) # 나온 결과에 활성화 함수 넣어준다
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')  
print(L1)

Tensor("MaxPool_1:0", shape=(?, 14, 14, 32), dtype=float32)


In [20]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
print(L2)

Tensor("MaxPool_4:0", shape=(?, 7, 7, 64), dtype=float32)


- 앞의 풀링 계층의 크기가 7 X 7 X 64 이므로, tf.reshape함수를 이용. 1차원으로 만듬.
- 인접한 계층의 모든 뉴런과 상호 연결된 계층을 완전 연결 계층(fully connect layer)라 한다
- 마지막 층의 뉴런수는 256개

In [18]:
W3 = tf.Variable(tf.random_normal([7*7*64,256], stddev=0.01))
L3 = tf.reshape(L2, [-1,7*7*64]) #fully connect layer로 연결시키기 위해 1차원으로 만들어줌
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
print(L3)

Tensor("Relu_5:0", shape=(?, 256), dtype=float32)


In [0]:
# 최종 출력값 L3 에서의 출력 256개를 입력값으로 받아서
# 0~9 레이블인 10개의 출력값을 만듭니다.
W4 = tf.Variable(tf.random_normal([256,10], stddev=0.01))
model = tf.matmul(L3,W4)

## 비용함수, 최적화 함수 구성

In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model,labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

## 신경망 모델 학습

In [0]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [0]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [0]:
import numpy as np

1 epoch (데이터 전체 학습 한번 )만 돌리기

In [36]:
%%time

total_cost = 0
epoch = 0

# 550번 학습

for i in range(total_batch):
  batch_xs, batch_ys = mnist.train.next_batch(batch_size) # batch_xs : 이미지 데이터 , batch_ys : 답(레이블) 데이터
  batch_xs = batch_xs.reshape(-1, 28, 28, 1) # 이미지 데이터를 CNN 모델을 위한 자료형태인 [28 28 1] 의 형태로 재구성합니다.
  _, cost_val = sess.run([optimizer, cost], feed_dict={X:batch_xs, Y: batch_ys})
  total_cost += cost_val
print('Epoch:', '%04d'% (epoch+1),'Avg.cost =', '{:3f}'.format(total_cost /total_batch)) #%04d - 0123

print('최적화 완료(1epoch)')


Epoch: 0001 Avg.cost = 0.324646
최적화 완료(1epoch)
CPU times: user 1min 52s, sys: 1.46 s, total: 1min 54s
Wall time: 59.2 s


In [0]:
# 실습 
# conv, pooling 3번
# 필터수 변경 32, 64
# epoch 20 